
# Задание 2




In [2]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('data.csv')

In [5]:
data.tail(5)

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
138710,5da3c1a0b09c,d3a29dbacfa1,Краснодарский край,Сочи,Недвижимость,Земельные участки,Продам,Поселений (ИЖС),NaN,Участок 7 сот. (ИЖС),Продается земельный участок в поселке Совет-Кв...,1700000.0,1738,2017-03-23,Shop,ecd17db17e628f94b2c5a5d608d3d0174153dc9167712b...,1285.0,0.12205
138711,192562b5fb5a,10c6a12cff32,Нижегородская область,Нижний Новгород,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),1111 Ока,"ВАЗ 1111 Ока, 2004",Состояние хорошее,60000.0,1,2017-03-15,Private,c767a6666ce0d9bcaa8a92ba126f7d2e93f4cd09c490c8...,1055.0,0.73760
138712,7177e036a9fe,5069d63c1e47,Калининградская область,Калининград,Для дома и дачи,Ремонт и строительство,Инструменты,NaN,NaN,Бензопила интерскол пцб 16\40л,"на запчасти,не заводится",1000.0,9,2017-03-15,Private,3952624a7d6f508cffde9b93816d65ca0b9a046f9848b4...,1243.0,0.20123
138713,8d3fe2371954,5e563726eb59,Самарская область,Самара,Для бизнеса,Готовый бизнес,Торговля,NaN,NaN,Свадебный салон именно для вас,Продажа готового бизнеса в преддверии свадебно...,200000.0,28,2017-03-18,Company,3c34a0a365c12d0186b37e8b6076e4f6a406fa5c47e993...,559.0,0.00000
138714,875a1dc9575a,b2b28b445fa4,Саратовская область,Саратов,Личные вещи,Товары для детей и игрушки,Детские коляски,NaN,NaN,Продам прогулочную коляску Corol,"Коляска легкая и крепкая, сезон зима-лето в хо...",4000.0,3,2017-03-20,Private,b3c67f05d7f7bfc3b4df66060af017d673a4d7421b5002...,999.0,0.00000


In [0]:
#data['description'].head(50)

In [0]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import LinearSVR

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [0]:
data['text_len'] = data['description'].apply(len)
data['text_len_tokens'] = data['description'].apply(lambda x: len(x.split()))
data['number_of_caps'] = data['description'].apply(lambda x: len([ch for ch in x if ch.isupper()]))
data['number_of_nonalphanum'] = data['description'].apply(lambda x: len([ch for ch in x if not ch.isalnum()]))
data['number_of_products'] = data['description'].apply(lambda x: len([ch for ch in x if ch == '+']))
data['used'] = data['description'].apply(lambda x: len([word for word in x.split() if word == 'б/у']))
data['excl'] = data['description'].apply(lambda x: len([ch for ch in x if ch == '!']))
data['condition'] = data['description'].apply(lambda x: len([word for word in x.split() if word == 'идеальном' or word == 'идеальное']))
data['eco'] = data['description'].apply(lambda x: len([word for word in x.split() if word == 'эко' or word == 'eco']))

data['number_of_products'] - количество плюсов. Чем больше плюсов, тем больше сопутсвующих товаров и аксессуаров, и тем, возможно, больше вероятность покупки.

data['used'] - б/у. Наверняка, на этот показатель покупатель тоже обращает внимание. 

data['excl'] - восклицательные знаки. Как я думаю, может уменьшать вероятность покупки. 

data['condition'] - говорит о состоянии продукта. Не факт, что влияет на вероятность (хотя должен), потому что "в идеальном состоянии" уже своего рода клише. 

data['eco'] - слова типа Эко-, эко или eco. Сейчас эта характеристика, вроде как, важна для покупателя (или, по крайней мере, важнее, чем раньше). 

In [0]:
X = data[['used', 'text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_products', 'excl', 'condition', 'eco']].values
y = data.deal_probability.values

In [14]:
X.shape

(138715, 9)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [16]:
regressor = Ridge(alpha=1)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.26335659420046176

Новые характеристики улучшают качество, но совсем уж незначительно (примерно 0.00002 процента). Возможно, больше влияют какие-то метахарактеристики (типа размера текста, наличия или отсутсвия латиницы и т.д.). Видимо, интуитивно понятная логика в таких задачах малоприменима. 

In [0]:
from sklearn.linear_model import SGDRegressor

In [0]:
from scipy.sparse import hstack

In [0]:
cv = CountVectorizer(max_features=500, min_df=3, max_df=0.1, ngram_range=(1,2))
tfidf = TfidfVectorizer(max_features=500, min_df=3,analyzer='char', ngram_range=(3,3), max_df=0.1)

X_cv = cv.fit_transform(data.description)
X_tfidf = tfidf.fit_transform(data.description)

X = hstack([X_cv, X_tfidf])

In [113]:
X_cv.shape, X_tfidf.shape

((138715, 500), (138715, 500))

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [115]:
regressor = SGDRegressor(max_iter=100, alpha=1)
regressor.fit(X_train, y_train)
preds = regressor.predict(X_test)
np.sqrt(mean_squared_error(y_test, preds))

0.2625618984489414

Качество, пусть и не сильно, но повысилось. У векторизаторов уменьшил максимальное количество фич (так как значимых показателей в таких текстах, думаю, не очень много, и не хочется переобучить), min_df (потому что могут быть опечатки, и хотя бы три случая одинакового написания говорят о том, что слово может быть значимым) и max_df (потому что слова, которые ни на что не влияют (продам, куплю), встречаются часто).

У регрессора увеличил количетво итераций в два раза и подкрутил регуляризацию (что, кстати, более заметно повлияло на качество). 

Общая точность повысилась до 0.26256

